In [56]:

import datetime
from torch._C import TensorType
from torch.utils.tensorboard import SummaryWriter
from uuid import uuid4 as uu
import shutil
import os
from tensorboard import program
import subprocess
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# plot tensorboard metrics
import os
import matplotlib.pyplot as plt
from tensorflow.python.summary.summary_iterator import summary_iterator


class TensorBoard:
    def __init__(self, tensor_board_root: str):
        self.tensor_board_root = tensor_board_root
        print('init writer')
        self.writer = SummaryWriter(tensor_board_root)

    def init_tensorboard(self):
        self.gen_session_id()
        self.launch_tensorboard()
        self.create_launchTensorboard_script()

    def launch_tensorboard(self):
        print('launch tensorboard')
      
        tensorboard_command = ["tensorboard", "--logdir", self.tensor_board_root]
        cmd = f"start cmd /k {' '.join(tensorboard_command)}"
        print(f"\nExecuting command: {cmd}")
        os.system(cmd)

    def create_launchTensorboard_script(self):
        # Create a script to launch tensorboard, place it in the tensorboard root
        script = f"tensorboard --logdir {self.tensor_board_root}"
        with open(os.path.join(self.tensor_board_root, 'launch_tensorboard.bat'), 'w') as f:
            f.write(script)

    def gen_session_id(self) -> str:
        day = str(datetime.date.today().day).rjust(2, '0')
        month = str(datetime.date.today().month).rjust(2, '0')
        year = str(datetime.date.today().year)[-2:]
        hour = str(datetime.datetime.now().hour).rjust(2, '0')
        minute = str(datetime.datetime.now().minute).rjust(2, '0')
        self.session_id = f'{day}-{month}-{year}_{hour}-{minute}_{str(uu())[-8:]}'
        print(f"Session ID: {self.session_id}")
        return self.session_id


    def write_board(self, epoch: int, train_loss: int, train_accuracy, train_recall, train_precision, test_loss, test_accuracy, test_recall, test_precision, current_learning_rate: float):
      
        # Calc metric and add scalars
        self.writer.add_scalars(f'Loss/{self.session_id}', {'Train': train_loss, 'Test': test_loss}, epoch)

        self.writer.add_scalars(f'Accuracy/{self.session_id}', {'Train': train_accuracy, 'Test': test_accuracy}, epoch)

        self.writer.add_scalars(f'Recall/{self.session_id}', {'Train': train_recall, 'Test': test_recall}, epoch)

        self.writer.add_scalars(f'Precision/{self.session_id}', {'Train': train_precision, 'Test': test_precision}, epoch)

        self.writer.add_scalar(f'Learning Rate/{self.session_id}', current_learning_rate, epoch)

        self.writer.flush()

    def write_confusion_matrix(self, epoch: int, confusion_matrix):
        # log confusion matrix

        plt.figure(figsize=(8, 6))
        sns.heatmap(confusion_matrix, annot=True, fmt='d', cmap='Blues')
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.title('Confusion Matrix')
        self.writer.add_figure('Confusion Matrix', plt.gcf(), global_step=epoch)
        self.writer.flush()

    def write_parameter_histogram(self, model, epoch):
        # log parameter distributions

        for name, param in model.named_parameters():
            plt.hist(param.flatten().detach().cpu().numpy(), bins=50, alpha=0.5, label=name)
            plt.legend()
            plt.title('Parameter Histograms')
            self.writer.add_figure('Parameter Histograms', plt.gcf(), global_step=epoch)
            self.writer.flush()




# ===== Plot TensorBoard Metrics =====


    def extract_tensorboard_data(self, log_dir=None):
        """
        Extracts data from TensorBoard log files.

        Args:
        - log_dir (str): Path to the directory containing TensorBoard event files.

        Returns:
        - data (dict): Dictionary containing metric data {metric_name: [(step1, value1), (step2, value2), ...]}.
        """
        if log_dir is None:
            log_dir = self.tensor_board_root
        data = {}
        events = {}

        for root, _, files in os.walk(log_dir):
            for file in files:
                if file.startswith("events.out"):
                    event_path = os.path.join(root, file)
                    for event in summary_iterator(event_path):
                        events[event.step] = event
                        for value in event.summary.value:
                            if value.tag not in data:
                                data[value.tag] = []
                            data[value.tag].append((event.step, value.simple_value))
        return data, events

    def plot_metrics(self, metric_values, label, title, save_path):
        """
        Plots and saves metrics as images.

        Args:
        - metrics (dict): Dictionary of metrics to plot.
        - save_path (str): Path to save the plot as an image.
        """
        plt.figure(figsize=(12, 8))
        steps, values = zip(*metric_values)
        plt.plot(steps, values, label=label, marker='o', markersize=3)
        plt.legend()
        plt.title(title)
        plt.xlabel('Step')
        plt.ylabel('Value')
        plt.tight_layout()
        plt.grid(True)
        #plt.savefig(f"{save_path}.png")
        plt.show()
        #plt.close()


    def plot_tensorboard_metrics(self, log_dir=None, title='TensorBoard Metrics', xlabel='Step', ylabel='Value', save_path='tensorboard_plot'):
        """
        Extracts and plots TensorBoard metrics as images.

        Args:
        - log_dir (str): Path to the directory containing TensorBoard event files.
        - title (str): Title of the plot.
        - xlabel (str): Label for the x-axis.
        - ylabel (str): Label for the y-axis.
        - save_path (str): Path to save the plot as an image.
        """
        if log_dir is None:
            log_dir = self.tensor_board_root
        metric_data = self.extract_tensorboard_data(log_dir)

        # Split into train and test metrics
        accuracy_values = [(step, value) for sublist in {metric: metric_data[metric] for metric in metric_data.keys() if 'Accuracy' in metric}.values() for step, value in sublist]
        loss_values = [(step, value) for sublist in {metric: metric_data[metric] for metric in metric_data.keys() if 'Loss' in metric}.values() for step, value in sublist]
        recall_values = [(step, value) for sublist in {metric: metric_data[metric] for metric in metric_data.keys() if 'Recall' in metric}.values() for step, value in sublist]
        precision_values = [(step, value) for sublist in {metric: metric_data[metric] for metric in metric_data.keys() if 'Precision' in metric}.values() for step, value in sublist]
        learning_rate_values = [(step, value) for sublist in {metric: metric_data[metric] for metric in metric_data.keys() if 'Learning Rate' in metric}.values() for step, value in sublist]
        
        self.plot_metrics(accuracy_values, 'Accuracy', title, save_path)
        # Call the function for each metric
        # self.plot_metrics(accuracies, 'accuracies_plot')
        # self.plot_metrics(losses, 'losses_plot')
        # self.plot_metrics(recalls, 'recalls_plot')
        # self.plot_metrics(precisions, 'precisions_plot')
        # self.plot_metrics(learning_rates, 'learning_rates_plot')
        
        return metric_data



In [ ]:
accuracy_values = [(step, value) for sublist in {metric: metric_data[metric] for metric in metric_data.keys() if 'Accuracy' in metric}.values() for step, value in sublist]
print(f'Acc Values: {accuracy_values}')

In [4]:
log_dir = "D:\\code\\Bachelorarbeit\\xx_code_BA\\trainingOutput\\2024-05-04_19-56-23_urinal_v2_resnet50\\tensorboard"


In [57]:
import src.trainCNN.tensorboardHandler as tbh
import json


log_dir = "D:\\code\\Bachelorarbeit\\xx_code_BA\\trainingOutput\\2024-05-04_19-56-23_urinal_v2_resnet50\\tensorboard"


tbh = TensorBoard(log_dir)


metric_data, events = tbh.extract_tensorboard_data(log_dir)

with open('metric_data.json', 'w') as f:
    json.dump(metric_data, f)
with open('events.json', 'w') as f:
    json.dump(events, f)

# metric_data = tbh.plot_tensorboard_metrics()

# # dumb metric data as json as file
# import json
# with open('metric_data.json', 'w') as f:
#     json.dump(metric_data, f)


init writer


TypeError: Object of type Event is not JSON serializable

In [9]:
import os
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import matplotlib.pyplot as plt

def plot_individual_metric_images_exact(logdir, output_dir):
    """
    Reads TensorBoard log files and saves separate plot images for each metric,
    combining both training and test data in one plot. Prints debug information.

    Parameters:
    - logdir: The directory where TensorBoard log files are located.
    - output_dir: The directory where plot images should be saved.
    """
    os.makedirs(output_dir, exist_ok=True)

    # Dictionary to accumulate metrics by their exact names
    metrics = {
        "Accuracy/Train": [],
        "Accuracy/Test": [],
        "Loss/Train": [],
        "Loss/Test": [],
        "Precision/Train": [],
        "Precision/Test": [],
        "Recall/Train": [],
        "Recall/Test": [],
    }

    # Traverse through event files and populate accumulators
    for root, _, files in os.walk(logdir):
        for file in files:
            if "events" in file:
                event_path = os.path.join(root, file)
                event_acc = EventAccumulator(event_path)
                event_acc.Reload()

                # List available scalar tags
                available_scalars = event_acc.Tags()["scalars"]
                print(f"Available scalars in {event_path}: {available_scalars}")

                # Collect the relevant metrics
                for tag in metrics:
                    if tag in available_scalars:
                        metrics[tag].extend(
                            [(s.step, s.value) for s in event_acc.Scalars(tag)]
                        )

    # Function to plot each metric, combining training and test, and handle empty lists gracefully
    def plot_metric(metric_name, train_tag, test_tag, title, ylabel):
        train_data = metrics[train_tag]
        test_data = metrics[test_tag]

        # Debugging output if lists are empty
        if not train_data:
            print(f"No training data found for {train_tag}")
        if not test_data:
            print(f"No test data found for {test_tag}")

        plt.figure()
        if train_data:
            plt.plot(*zip(*sorted(train_data)), label="Training", color='blue')
        if test_data:
            plt.plot(*zip(*sorted(test_data)), label="Test", color='orange')

        plt.title(title)
        plt.legend()
        plt.xlabel("Steps")
        plt.ylabel(ylabel)

        # Save the plot as an image
        filename = os.path.join(output_dir, f"{metric_name}.png")
        plt.savefig(filename)
        plt.close()

    # Plot each of the following metrics as separate images
    plot_metric("accuracy", "Accuracy/Train", "Accuracy/Test", "Accuracy", "Accuracy")
    plot_metric("loss", "Loss/Train", "Loss/Test", "Loss", "Loss")
    plot_metric("precision", "Precision/Train", "Precision/Test", "Precision", "Precision")
    plot_metric("recall", "Recall/Train", "Recall/Test", "Recall", "Recall")


In [10]:
plot_individual_metric_images_exact(log_dir, log_dir)

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


Available scalars in D:\code\Bachelorarbeit\xx_code_BA\trainingOutput\2024-05-04_19-56-23_urinal_v2_resnet50\tensorboard\events.out.tfevents.1714845391.mts_workstation.40156.0: ['Learning Rate/04-05-24_19-56_34c89e41']
Available scalars in D:\code\Bachelorarbeit\xx_code_BA\trainingOutput\2024-05-04_19-56-23_urinal_v2_resnet50\tensorboard\events.out.tfevents.1714851415.mts_workstation.21924.0: []
Available scalars in D:\code\Bachelorarbeit\xx_code_BA\trainingOutput\2024-05-04_19-56-23_urinal_v2_resnet50\tensorboard\events.out.tfevents.1714852888.mts_workstation.21924.1: []
Available scalars in D:\code\Bachelorarbeit\xx_code_BA\trainingOutput\2024-05-04_19-56-23_urinal_v2_resnet50\tensorboard\events.out.tfevents.1714853052.mts_workstation.21924.2: []
Available scalars in D:\code\Bachelorarbeit\xx_code_BA\trainingOutput\2024-05-04_19-56-23_urinal_v2_resnet50\tensorboard\events.out.tfevents.1714853061.mts_workstation.21924.3: []
Available scalars in D:\code\Bachelorarbeit\xx_code_BA\traini

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


No training data found for Precision/Train
No test data found for Precision/Test
No training data found for Recall/Train
No test data found for Recall/Test


In [12]:
import src.mts_utils as mts
import time

starttime = time.time()

time.sleep(5)

print(mts.get_TimeElapsed(starttime))

00:00:05.00
